In [1]:
from files import generate_tone, compute_group_delay, compute_phase_spectrum
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import soundfile as sf
import os

# ============================================================================
# SECTION 0: SETUP AND SIGNAL GENERATION
# ============================================================================
# Why: Create synthetic audio signals for advanced frequency domain analysis.
# What we'll build: Two pure tones at different frequencies + noise
# Key difference from basic.ipynb: We'll focus on phase and group delay analysis
# ============================================================================


# --- Setup: Key audio parameters ---
duration = 3  # seconds
sample_rate = 16000  # Hz (samples per second). Standard for speech/phone audio.

# --- Create two component signals ---
# Low tone (like AC hum): 150 Hz - slow oscillations
# High tone (like birdsong): 3000 Hz - fast oscillations
t, tone_low = generate_tone(150, duration, sample_rate)
_, tone_high = generate_tone(3000, duration, sample_rate)
clean_audio = tone_low + tone_high  # Mix them together (linear superposition)

# Use real audio
# audio_path = os.path.expanduser("~/Downloads/biluma.wav")  # replace with your file
# clean_audio, sample_rate = sf.read(audio_path)

# --- Add realistic noise ---
# Why: Real recordings are never clean. Noise affects phase coherence and timing estimates.
noise = np.random.normal(0, 0.01, clean_audio.shape)  # Gaussian noise: mean=0, std=0.1
noisy_audio = clean_audio + noise  # Composite signal: tone_low + tone_high + noise

print(f"Signal Generated. shape: {noisy_audio.shape}, Sampling Rate: {sample_rate}")
print(f"  → {noisy_audio.shape[0]} samples at {sample_rate} Hz = {duration} seconds")

Signal Generated. shape: (48000,), Sampling Rate: 16000
  → 48000 samples at 16000 Hz = 3 seconds


In [2]:
# ============================================================================
# OPTIONAL: LOAD REAL AUDIO FILE
# ============================================================================
# Uncomment below to load your own audio file instead of synthetic signal
# ============================================================================

# import soundfile as sf
# import os

# audio_path = os.path.expanduser("~/Downloads/biluma.wav")  # replace with your file
# noisy_audio, sample_rate = sf.read(audio_path)

# if noisy_audio.ndim > 1:
#     noisy_audio = noisy_audio.mean(axis=1)  # Convert stereo to mono if needed

============================================================================
SECTION 1: PHASE SPECTRUM ANALYSIS
============================================================================
Why: Phase contains timing information about frequency components.
     While magnitude tells us "what" frequencies exist, phase tells us "when"
     they occur relative to each other.
What we'll see: Phase values in radians, showing timing relationships between
                frequency components.
Key insight: Only bins near 150 Hz and 3000 Hz have strong magnitude.
             Everywhere else, magnitude is dominated by noise.
             For noise-dominated bins, phase becomes uniformly random in [-π, π].



### Compute Phase Spectrum
How: FFT decomposes signal into frequency components, each with magnitude and phase
Math: X[k] = |X[k]| * e^(j*φ[k]) where φ[k] is the phase

Phase wrapping:
  - np.angle() gives phase in [-π, π] (wrapped)
  - If phase jumps by more than π, assume it's a wrap and add/subtract 2π
  - Formula: φ_unwrap(k) = φ(k) + 2π*n_k
  - Use np.unwrap() to study delay or system behavior

Optional masking:
  - Mask phase values where magnitude is below threshold
  - Focus on strong frequency components (e.g., top 10% by magnitude)

============================================================================

In [3]:
freqs, phase, magnitude = compute_phase_spectrum(
    noisy_audio, 
    sample_rate, 
    unwrap=True,  # Unwrap phase to remove 2π discontinuities
    mask_threshold=None  # Set to np.percentile(magnitude, 90) to mask weak components
)

# Optional: Compute masked phase for comparison
threshold = np.percentile(magnitude, 90)  # Keep strongest 10%
_, phase_masked, _ = compute_phase_spectrum(
    noisy_audio, 
    sample_rate, 
    unwrap=True,
    mask_threshold=threshold
)

# --- Interactive Visualization ---
fig = go.Figure()

# Full phase spectrum (all frequencies)
fig.add_trace(go.Scatter(
    x=freqs, 
    y=phase, 
    mode="lines", 
    name="Phase Spectrum (Unwrapped)",
    line=dict(color='blue', width=1)
))

# Optional: Add masked version to compare
fig.add_trace(go.Scatter(
    x=freqs, 
    y=phase_masked, 
    mode="lines", 
    name="Phase (Masked: Top 10%)",
    line=dict(color='red', width=1, dash='dash')
))

fig.update_layout(
    title="Phase Spectrum: Timing Information in Frequency Domain",
    xaxis_title="Frequency (Hz)",
    yaxis_title="Phase (radians)",
    template="plotly_white",
    width=1000,
    height=400,
    hovermode='x unified'
)

fig.show()

print(f"Phase Spectrum computed:")
print(f"  → {len(freqs)} frequency bins from 0 Hz to {sample_rate/2:.0f} Hz")
print(f"  → Phase range: [{np.nanmin(phase):.2f}, {np.nanmax(phase):.2f}] radians")
print(f"  → Strong components at 150 Hz and 3000 Hz have stable phase")
print(f"  → Noise-dominated bins show random phase in [-π, π]")

Phase Spectrum computed:
  → 24001 frequency bins from 0 Hz to 8000 Hz
  → Phase range: [-244.37, 46.32] radians
  → Strong components at 150 Hz and 3000 Hz have stable phase
  → Noise-dominated bins show random phase in [-π, π]


============================================================================
SECTION 2: GROUP DELAY – CLEAN vs NOISY + DIFFERENCE (LINKED SUBPLOTS)
============================================================================
Goal: Combine the ideas from the previous group-delay sections into a single
      compact view with two linked subplots:
        - Left:  Group delay of clean vs noisy (windowed + smoothed)
        - Right: Group delay difference (noisy - clean) to quantify noise impact

Why group delay?
  - Group delay measures frequency-dependent timing distortion.
  - It answers: "Does my processing introduce frequency-dependent timing shifts?"
  - For signals, jagged/unstable group delay indicates unreliable timing estimates (often due to noise).

Clean vs Noisy (left subplot):
  - Clean signal: tone_low + tone_high (150 Hz + 3000 Hz) with stable timing.
  - Noisy signal: clean + Gaussian noise → perturbed timing, reduced phase
    coherence.
  - Windowing (Hanning) smooths the start/end to remove artificial boundary
    effects before FFT.
  - Phase smoothing (Savitzky–Golay) reduces noise in the group delay curve.

Difference plot (right subplot):
  - Δτ_g(ω) = τ_g_noisy(ω) - τ_g_clean(ω)
  - Δτ_g ≈ 0  → noise has little timing impact at that frequency.
  - Large |Δτ_g| → noise strongly perturbs timing / phase coherence.
  - Positive Δτ_g: noisy signal appears delayed vs clean.
  - Negative Δτ_g: noisy signal appears advanced vs clean.
  
============================================================================

In [ ]:
# ---------------------------------------------------------------------------
# 1. Prepare clean & noisy signals for group delay computation
# ---------------------------------------------------------------------------

# Windowing: smooth fade-in/out to avoid artificial edges before FFT
window = np.hanning(len(clean_audio))

# Compute group delay with phase smoothing for both signals
freqs, gd_clean = compute_group_delay(
    clean_audio * window,
    sample_rate, 
    smooth_phase=True,   # Smooth phase to reduce noise artifacts
    window_length=101,
    polyorder=3,
)

_, gd_noisy = compute_group_delay(
    noisy_audio * window,
    sample_rate,
    smooth_phase=True,
    window_length=101,
    polyorder=3,
)

# Group delay difference: quantify noise impact
gd_diff = gd_noisy - gd_clean

# ---------------------------------------------------------------------------
# 2. Build linked side-by-side subplots (clean vs noisy, and difference)
# ---------------------------------------------------------------------------

fig = make_subplots(
    rows=1,
    cols=2,
    shared_xaxes=True,   # Share frequency axis between left & right
    shared_yaxes=True,   # Share group-delay axis between left & right
    horizontal_spacing=0.08,
    subplot_titles=[
        "Group Delay: Clean vs Noisy (Windowed + Smoothed)",
        "Group Delay Difference: Noisy − Clean (Noise Impact)",
    ],
)

# Left subplot: clean vs noisy group delay
fig.add_trace(
    go.Scatter(
        x=freqs,
        y=gd_clean,
        name="Clean (150 + 3000 Hz)",
        line=dict(color="green", width=2),
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        x=freqs,
        y=gd_noisy,
        name="Noisy (150 + 3000 Hz + noise)",
        line=dict(color="red", width=2),
    ),
    row=1,
    col=1,
)

# Right subplot: group delay difference (noisy - clean)
fig.add_trace(
    go.Scatter(
        x=freqs,
        y=gd_diff,
        name="Noisy − Clean",
        line=dict(color="purple", width=2),
        fill="tozeroy",
        fillcolor="rgba(128, 0, 128, 0.18)",
    ),
    row=1,
    col=2,
)

# Horizontal reference line at zero in the right subplot
fig.add_hline(
    y=0, line_dash="dash", line_color="gray",
    opacity=0.5,
    annotation_text="No difference",
    row=1, col=2
)

# ---------------------------------------------------------------------------
# 3. Axis labelling, linking, and default view
# ---------------------------------------------------------------------------

# Shared axis labels
fig.update_xaxes(title_text="Frequency (Hz)", row=1, col=1)
fig.update_xaxes(title_text="Frequency (Hz)", row=1, col=2)
fig.update_yaxes(title_text="Group Delay / Δ Group Delay (seconds)", row=1, col=1)

fig.update_layout(
    title="Group Delay: Clean vs Noisy and Noise Impact (Linked Axes)",
    height=600,
    width=1400,
    template="plotly_white",
    hovermode="x unified",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1.0),
)

# Ensure interactive linking of x & y ranges between the two subplots
# (zoom/pan in one subplot updates the other as well)
fig.update_xaxes(matches="x")
fig.update_yaxes(matches="y")

# Focus by default on the low-frequency band where structure is most interpretable
fig.update_xaxes(range=[0, 4500])
fig.update_yaxes(range=[-1, 1])

fig.show()

print("Group delay analysis:")
print("  → Left:  Clean vs noisy group delay (windowed + phase-smoothed)")
print("  → Right: Noise-induced group delay difference (noisy − clean)")
print(f"Group Delay computed:")
print(
    f"  → Group delay range Noisy: [{np.min(gd_noisy):.4f}, {np.max(gd_noisy):.4f}] seconds")
print(f"  → Group delay range Clean: [{np.min(gd_clean):.4f}, {np.max(gd_clean):.4f}] seconds")
print(f"  → Jagged appearance indicates timing estimate unreliability")
print(f"  → Noise makes interpretation difficult at each frequency")
print(f"Noise Impact Analysis:")
print(f"  → Mean absolute difference: {np.mean(np.abs(gd_diff)):.6f} seconds")
print(
    f"  → Max difference: {np.max(np.abs(gd_diff)):.6f} seconds at {freqs[np.argmax(np.abs(gd_diff))]:.1f} Hz"
)
print(
    f"  → Large differences indicate frequencies where noise degrades timing estimates"
)


Group delay analysis:
  → Left:  Clean vs noisy group delay (windowed + phase-smoothed)
  → Right: Noise-induced group delay difference (noisy − clean)
Group Delay computed:
  → Group delay range Noisy: [-0.6995, 0.7418] seconds
  → Group delay range Clean: [-0.0678, 0.1335] seconds
  → Jagged appearance indicates timing estimate unreliability
  → Noise makes interpretation difficult at each frequency
Noise Impact Analysis:
  → Mean absolute difference: 0.162717 seconds
  → Max difference: 0.741816 seconds at 3136.3 Hz
  → Large differences indicate frequencies where noise degrades timing estimates


============================================================================
SECTION 3: MAGNITUDE-SQUARED COHERENCE (CLEAN vs NOISY)
============================================================================
Why coherence?
  - Measures how linearly predictable one signal is from another at each frequency.
  - Values near 1 → clean and noisy share phase+amplitude structure; near 0 → structure is decorrelated.

How it works:
  - Compute cross-spectrum C_xy(f) and auto-spectra P_xx(f), P_yy(f) with Welch averaging.
  - Coherence = |C_xy|^2 / (P_xx P_yy) ∈ [0,1]; normalization removes absolute gain differences.
  - We mask bins where clean P_xx is tiny to avoid interpreting noise-only regions.

Why it works:
  - High coherence implies stable phase relationship and shared energy over time.
  - Even if noise raises the floor, consistent speech structure keeps cross-spectrum aligned.
  - Low coherence signals structural damage (nonlinear distortion, heavy reverberation, or interfering talkers).

Where it's useful (real-world):
  - Multi-mic arrays: check mic alignment, detect failed or mis-positioned capsules, steer beamformers.
  - Recording chains: verify analog + digital paths stay phase-aligned across channels.
  - Robustness checks: locate frequency bands where denoising or ASR will struggle due to decorrelation.

What to look for here:
  - Broad high-coherence band → speech structure preserved.
  - Low/no coherence in speech band → broadband noise or structural corruption.
  - Patchy coherence → reverberation or narrowband interferers.

============================================================================


In [7]:
import numpy as np
from scipy import signal
import plotly.graph_objects as go

# ---------------------------------------------------------------------------
# Coherence pipeline: measure linear predictability of noisy_audio from clean_audio
# Steps:
#   1) Compute coherence + clean power for a speech-aware mask
#   2) Extract interpretable features over speech-supported bins
#   3) Visualize coherence and print a rule-based diagnosis
# ---------------------------------------------------------------------------


def compute_coherence(clean_audio, noisy_audio, sample_rate):
    """
    Computes magnitude-squared coherence and clean power spectrum.

    Coherence: linear predictability of noisy_audio from clean_audio
    Pxx: used to ignore frequency bins where clean signal has no energy
    """

    # Ensure equal length so Welch segments align
    min_len = min(len(clean_audio), len(noisy_audio))
    clean_audio = clean_audio[:min_len]
    noisy_audio = noisy_audio[:min_len]

    # Magnitude-squared coherence with Welch averaging (Hann window, 50% overlap)
    freqs, coh = signal.coherence(
        clean_audio,
        noisy_audio,
        fs=sample_rate,
        nperseg=1024,
        noverlap=512,
        window="hann",
    )

    # Clean-signal power spectrum for gating out noise-only bins
    _, Pxx = signal.welch(
        clean_audio, fs=sample_rate, nperseg=1024, noverlap=512, window="hann"
    )

    return freqs, coh, Pxx


def extract_coherence_features(freqs, coh, Pxx):
    """
    Extracts interpretable coherence features only where speech exists.
    """

    # Speech-dominant band for typical voice content
    speech_band = (freqs >= 100) & (freqs <= 4000)

    # Ignore bins with negligible clean-signal energy
    power_threshold = 0.01 * np.max(Pxx)
    valid_bins = speech_band & (Pxx > power_threshold)

    if not np.any(valid_bins):
        return {
            "mean": 0.0,
            "median": 0.0,
            "std": 0.0,
            "low_fraction": 1.0,
            "high_fraction": 0.0,
            "valid_bins": valid_bins,
        }

    coh_valid = coh[valid_bins]

    return {
        "mean": float(np.mean(coh_valid)),
        "median": float(np.median(coh_valid)),
        "std": float(np.std(coh_valid)),
        "low_fraction": float(np.mean(coh_valid < 0.2)),
        "high_fraction": float(np.mean(coh_valid > 0.6)),
        "valid_bins": valid_bins,
    }


def plot_coherence(freqs, coh, Pxx, valid_bins):
    """
    Plots coherence and highlights frequency regions where speech exists.

    Blue  : coherence curve
    Green : bins where clean speech has energy and coherence is meaningful
    """

    fig = go.Figure()

    # Full coherence curve
    fig.add_trace(
        go.Scatter(
            x=freqs,
            y=coh,
            mode="lines",
            name="Coherence (Clean vs Noisy)",
            line=dict(color="royalblue"),
        )
    )

    # Highlight bins where clean signal has energy (speech-supported bins)
    fig.add_trace(
        go.Scatter(
            x=freqs[valid_bins],
            y=coh[valid_bins],
            mode="markers",
            name="Speech-supported bins",
            marker=dict(color="green", size=5),
        )
    )

    fig.update_layout(
        title="Magnitude-Squared Coherence with Speech-Supported Frequency Mask",
        xaxis_title="Frequency (Hz)",
        yaxis_title="Coherence (0 to 1)",
        yaxis=dict(range=[0, 1]),
        template="plotly_white",
    )

    fig.show()


def interpret_coherence(features):
    """
    Diagnoses degradation type using coherence structure.
    """

    print("=== Coherence-Based Structural Diagnosis ===")

    print(f"Mean coherence (speech-supported): {features['mean']:.2f}")
    print(f"Median coherence: {features['median']:.2f}")
    print(f"High-coherence fraction (>0.6): {features['high_fraction']:.2f}")
    print(f"Low-coherence fraction (<0.2): {features['low_fraction']:.2f}")
    print(f"Spectral variability (std): {features['std']:.2f}")

    # Simple rule-based diagnosis for illustrative degradation categories
    if (
        features["mean"] < 0.3
        and features["std"] < 0.15
        and features["low_fraction"] > 0.6
    ):
        print("Diagnosis: Additive broadband noise.")
        print("• Speech structure is present but masked.")
        print("• Low coherence is expected here.")
        print("• Classical and neural denoising are appropriate.")
        return

    if features["high_fraction"] > 0.4 and features["median"] > 0.5:
        print("Diagnosis: Speech structure largely preserved.")
        print("• Noise is weak, correlated, or spectrally limited.")
        print("• ASR and downstream tasks should perform well.")
        return

    if (
        features["mean"] < 0.2
        and features["std"] > 0.25
        and features["high_fraction"] < 0.1
    ):
        print("Diagnosis: Structural or nonlinear speech damage.")
        print("• Speech is not linearly recoverable.")
        print("• Classical denoising will fail.")
        return

    print("Diagnosis: Mixed or ambiguous degradation.")
    print("• Combine coherence with SNR, SI-SNR, or modulation features.")


# --- Run the coherence analysis end-to-end ---
freqs, coh, Pxx = compute_coherence(clean_audio, noisy_audio, sample_rate)

features = extract_coherence_features(freqs, coh, Pxx)

plot_coherence(freqs, coh, Pxx, features["valid_bins"])

interpret_coherence(features)


=== Coherence-Based Structural Diagnosis ===
Mean coherence (speech-supported): 1.00
Median coherence: 1.00
High-coherence fraction (>0.6): 1.00
Low-coherence fraction (<0.2): 0.00
Spectral variability (std): 0.00
Diagnosis: Speech structure largely preserved.
• Noise is weak, correlated, or spectrally limited.
• ASR and downstream tasks should perform well.
